## Using sqlite3 with Python

In [1]:
# lets import some modules, shall we?
import sqlite3
import pandas as pd

There are two ways to do this, the long and ugly native way below:

In [2]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
cur = con.cursor()
# change query
query = "SELECT r.name, SUM(r.revenue) as total_revenue FROM (SELECT f.name, CASE WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END AS revenue FROM Bookings b LEFT JOIN Facilities f ON b.facid = f.facid) r GROUP BY r.name HAVING SUM(r.revenue) < 1000 ORDER BY SUM(r.revenue);"
cur.execute(query)
rows = cur.fetchall()
# get column names
names = [description[0] for description in cur.description]
print(names)
# get results by row
for row in rows:
    print(row)
con.close()

['name', 'total_revenue']
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


And the way of importing the output directly into a pandas dataframe that I prefer to use for this assignment after this point.

#### Query 10: 
Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [3]:
# I prefer to read into Pandas for nicer look
query = "SELECT r.name, SUM(r.revenue) as total_revenue FROM (SELECT f.name, CASE WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END AS revenue FROM Bookings b LEFT JOIN Facilities f ON b.facid = f.facid) r GROUP BY r.name HAVING SUM(r.revenue) < 1000 ORDER BY SUM(r.revenue);"
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
df = pd.read_sql_query(query, con)
con.close()
df

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


#### Query 11:
Produce a report of members and who recommended them in alphabetic surname,firstname order

In [4]:
query = "SELECT DISTINCT m1.surname, m1.firstname, m2.firstname || ' ' || m2.surname as recommended_by FROM Members m1 LEFT JOIN Members m2 ON m1.recommendedby = m2.memid WHERE m1.surname <> 'GUEST' AND m2.surname <> 'GUEST' ORDER BY m1.surname, m1.firstname;"
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
df = pd.read_sql_query(query, con)
con.close()
df

,surname,firstname,recommended_by
0,Bader,Florence,Ponder Stibbons
1,Baker,Anne,Ponder Stibbons
2,Baker,Timothy,Jemima Farrell
3,Boothe,Tim,Tim Rownam
4,Butters,Gerald,Darren Smith
5,Coplin,Joan,Timothy Baker
6,Crumpet,Erica,Tracy Smith
7,Dare,Nancy,Janice Joplette
8,Genting,Matthew,Gerald Butters
9,Hunt,John,Millicent Purview


#### Query 12:
Find the facilities with their usage by member, but not guests

In [5]:
query = "SELECT f.name as facility, m.firstname || ' ' || m.surname as member, ROUND(SUM(b.slots)/2, 1) as hours_booked FROM Bookings b LEFT JOIN Facilities f ON b.facid = f.facid LEFT JOIN Members m ON b.memid = m.memid WHERE b.memid <> 0 GROUP BY f.name, m.firstname || ' ' || m.surname;"
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
df = pd.read_sql_query(query, con)
con.close()
df

,facility,member,hours_booked
0,Badminton Court,Anna Mackenzie,48.0
1,Badminton Court,Anne Baker,15.0
2,Badminton Court,Burton Tracy,3.0
3,Badminton Court,Charles Owen,9.0
4,Badminton Court,Darren Smith,216.0
...,...,...,...
197,Tennis Court 2,Ramnaresh Sarwin,18.0
198,Tennis Court 2,Tim Boothe,84.0
199,Tennis Court 2,Tim Rownam,9.0
200,Tennis Court 2,Timothy Baker,10.0


#### Query 13:
Find the facilities usage by month, but not guests

In [6]:
query = "SELECT f.name as facility, strftime('%m', starttime) as month, ROUND(SUM(b.slots)/2, 1) as hours_booked FROM Bookings b LEFT JOIN Facilities f ON b.facid = f.facid WHERE b.memid <> 0 GROUP BY f.name, strftime('%m', starttime);"
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
df = pd.read_sql_query(query, con)
con.close()
df

,facility,month,hours_booked
0,Badminton Court,07,82.0
1,Badminton Court,08,207.0
2,Badminton Court,09,253.0
3,Massage Room 1,07,83.0
4,Massage Room 1,08,158.0
5,Massage Room 1,09,201.0
6,Massage Room 2,07,4.0
7,Massage Room 2,08,9.0
8,Massage Room 2,09,14.0
9,Pool Table,07,55.0
